# Leitura de Dados

In [1]:
import pandas as pd

In [2]:
dados = pd.read_excel("Dados.xlsx", sheet_name="Dados", index_col=0)
dados

,Atlanta,Chicago,NY,Dallas,Capacidade
Planta 1,130,239,256,120,400.0
Planta 2,225,161,236,288,650.0
Planta 3,190,123,153,198,500.0
Demanda,350,400,450,250,NaN


In [5]:
dados.Capacidade.drop("Demanda")

Planta 1    400.0
Planta 2    650.0
Planta 3    500.0
Name: Capacidade, dtype: float64

In [7]:
dados_capacidades = dados.Capacidade.drop("Demanda")
dados_capacidades

Planta 1    400.0
Planta 2    650.0
Planta 3    500.0
Name: Capacidade, dtype: float64

In [8]:
dados_demandas = dados.loc["Demanda"].drop("Capacidade")
dados_demandas

Atlanta    350.0
Chicago    400.0
NY         450.0
Dallas     250.0
Name: Demanda, dtype: float64

In [10]:
print(dados_capacidades)

Planta 1    400.0
Planta 2    650.0
Planta 3    500.0
Name: Capacidade, dtype: float64


In [11]:
dados_custos = dados.iloc[:-1,:-1]
dados_custos

,Atlanta,Chicago,NY,Dallas
Planta 1,130,239,256,120
Planta 2,225,161,236,288
Planta 3,190,123,153,198


# Importando o pyomo

In [12]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [13]:
modelo = pyo.ConcreteModel()

### Criando Índices

In [14]:
modelo.i = Set(initialize= dados_custos.index, doc='Plantas')
modelo.j = Set(initialize= dados_custos.columns, doc = "Cidades")

### Criando os parâmetros

In [15]:
modelo.capacidades = Param (modelo.i, initialize= dados_capacidades, doc='Capacidades')
modelo.demandas = Param(modelo.j, initialize=dados_demandas, doc='Demandas')

In [17]:
dicionario_custos = dados_custos.stack().to_dict()
dicionario_custos

{('Planta 1', 'Atlanta'): 130,
 ('Planta 1', 'Chicago'): 239,
 ('Planta 1', 'NY'): 256,
 ('Planta 1', 'Dallas'): 120,
 ('Planta 2', 'Atlanta'): 225,
 ('Planta 2', 'Chicago'): 161,
 ('Planta 2', 'NY'): 236,
 ('Planta 2', 'Dallas'): 288,
 ('Planta 3', 'Atlanta'): 190,
 ('Planta 3', 'Chicago'): 123,
 ('Planta 3', 'NY'): 153,
 ('Planta 3', 'Dallas'): 198}

In [18]:
modelo.custos = Param(modelo.i, modelo.j, initialize = dicionario_custos, doc='Custos')

### Criando as Variáveis

In [19]:
modelo.x = Var(modelo.i, modelo.j, bounds=(0.0,None), doc='Quantidades Enviadas')

### Criando as Restrições

In [20]:
def regra_capacidade(modelo, i):
    return sum(modelo.x[i,j] for j in modelo.j) <= modelo.capacidades[i]

def regra_demanda(modelo, j):
    return sum(modelo.x[i,j] for i in modelo.i) >= modelo.demandas[j]


modelo.restr_capacidade = Constraint(modelo.i, rule = regra_capacidade)
modelo.restr_demanda = Constraint(modelo.j, rule = regra_demanda)



### Função Objetivo

In [21]:
def custo_total(modelo):
    return sum(modelo.custos[i,j] * modelo.x[i,j] for i in modelo.i for j in modelo.j)

modelo.Z = Objective(rule=custo_total, sense=minimize) 


In [22]:
opt = SolverFactory("gurobi")
opt.solve(modelo)

{'Problem': [{'Name': 'x13', 'Lower bound': 225850.0, 'Upper bound': 225850.0, 'Number of objectives': 1, 'Number of constraints': 8, 'Number of variables': 13, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of continuous variables': 13, 'Number of nonzeros': 25, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.06838417053222656', 'Error rc': 0, 'Time': 1.154653787612915}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [23]:
for v in modelo.component_data_objects(Var):
    print (str(v), v.value)
    
print("O valor ótimo encontrado foi: R$", modelo.Z())

x[Planta 1,Atlanta] 150.0
x[Planta 1,Chicago] 0.0
x[Planta 1,NY] 0.0
x[Planta 1,Dallas] 250.0
x[Planta 2,Atlanta] 200.0
x[Planta 2,Chicago] 350.0
x[Planta 2,NY] 0.0
x[Planta 2,Dallas] 0.0
x[Planta 3,Atlanta] 0.0
x[Planta 3,Chicago] 50.0
x[Planta 3,NY] 450.0
x[Planta 3,Dallas] 0.0
O valor ótimo encontrado foi: R$ 225850.0
